"""

REFER SCRIPT yoloToPhoscPipeLine2.ipynb 

"""

In [1]:
import os
import cv2
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
import shutil
import numpy as np
import tensorflow as tf

#print("\n\t tf:",tf.__version__)
#tf.enable_eager_execution()
# from tensorflow.keras.utils import plot_model
#from yolov3.dataset import Dataset
from yolov3.yolov4 import compute_loss1
from yolov3.utils import load_yolo_weights
from yolov3.configs import *
from evaluate_mAP import get_mAP

print("\n\t YOLO_TYPE:",YOLO_TYPE)

import os
import cv2
import random
import numpy as np
import tensorflow as tf
import pickle
from yolov3.utils import read_class_names, image_preprocess#,draw_bbox
from yolov3.yolov3 import bbox_iou
from yolov3.configs import *
from yolov3.utils import load_yolo_weights, detect_image,  nms, read_class_names #image_preprocesss,
from supportFunctions import draw_bbox, postprocess_boxes

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from yoloModify import yolo33

'''
    more details regarding Dataset1 is in yolo3PhoscModify2.ipynb
'''

from yolov3.dataset import Dataset1
from yolov3.yolov3 import Create_Yolov3
global TRAIN_FROM_CHECKPOINT

gpus = tf.config.experimental.list_physical_devices('GPU')
print(f'GPUs {gpus}')
if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

#trainset = Dataset1('train')
testset = Dataset1('test')

if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

steps_per_epoch = len(testset)
global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
warmup_steps = TRAIN_WARMUP_EPOCHS * steps_per_epoch
total_steps = TRAIN_EPOCHS * steps_per_epoch




2022-03-30 23:14:36.212625: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/aniketag/.conda/envs/TensorFlow-2.x-YOLOv3/lib/python3.8/site-packages/cv2/../../lib64:
2022-03-30 23:14:36.212653: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7497765082582405735
]

	 YOLO_TYPE: yolov3


2022-03-30 23:14:37.614486: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-30 23:14:37.615478: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/aniketag/.conda/envs/TensorFlow-2.x-YOLOv3/lib/python3.8/site-packages/cv2/../../lib64:
2022-03-30 23:14:37.615501: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-30 23:14:37.615544: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (aniketag-Latitude-7420): /proc/driver/nvidia/ver

GPUs []

	 self.num_classes= 1
100


[ WARN:0@2.796] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/g06-018o.png'): can't open/read file: check file path/integrity
[ WARN:0@2.865] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/k07-059a.png'): can't open/read file: check file path/integrity
[ WARN:0@2.959] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/a05-121.png'): can't open/read file: check file path/integrity
[ WARN:0@3.034] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/r06-103.png'): can't open/read file: check file path/integrity
[ WARN:0@3.086] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/h01-014.png'): can't open/read file: check file path/integrity
[ WARN:0

[ WARN:0@6.225] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/m02-048.png'): can't open/read file: check file path/integrity
[ WARN:0@6.286] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/k02-112.png'): can't open/read file: check file path/integrity
[ WARN:0@6.347] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/g06-050i.png'): can't open/read file: check file path/integrity
[ WARN:0@6.441] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/g01-043.png'): can't open/read file: check file path/integrity
[ WARN:0@6.542] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/n02-146.png'): can't open/read file: check file path/integrity
[ WARN:0@

[ WARN:0@9.885] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/h01-004.png'): can't open/read file: check file path/integrity
[ WARN:0@10.013] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/f07-019a.png'): can't open/read file: check file path/integrity
[ WARN:0@10.109] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/k07-134.png'): can't open/read file: check file path/integrity
[ WARN:0@10.231] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/a04-059.png'): can't open/read file: check file path/integrity
[ WARN:0@10.278] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/g06-047k.png'): can't open/read file: check file path/integrity
[ WA

In [2]:
#from yolov3.yolov4 import Create_YoloPhosc
TRAIN_MODEL_NAME='yolov3_custom'
TRAIN_MODEL_NAME

if YOLO_TYPE == "yolov3":
    Darknet_weights = YOLO_V3_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V3_WEIGHTS


if TRAIN_TRANSFER:
    #Darknet = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=YOLO_COCO_CLASSES)
    Darknet = Create_Yolov3(input_size=YOLO_INPUT_SIZE, CLASSES=YOLO_COCO_CLASSES)

    #print("Darknet_weights=",os.path.isfile(Darknet_weights))
    load_yolo_weights(Darknet, Darknet_weights) # use darknet weights
    
#yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)

#yolo=Create_Yolov3(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)
yolo=yolo33(input_size=416, channels=3, training=True, CLASSES=YOLO_COCO_CLASSES)

TRAIN_FROM_CHECKPOINT=True
if TRAIN_FROM_CHECKPOINT:
    try:
        print("\n\t load weights!!!")
        yolo.load_weights(f"./checkpoints/{TRAIN_MODEL_NAME}")
        print("\n\t loading complete")

    except ValueError:
        print("Shapes are incompatible, transfering Darknet weights")
        TRAIN_FROM_CHECKPOINT = False
'''
if TRAIN_TRANSFER and not TRAIN_FROM_CHECKPOINT:
    for i, l in enumerate(Darknet.layers):
        layer_weights = l.get_weights()
        if layer_weights != []:
            try:
                yolo.layers[i].set_weights(layer_weights)
            except:
                print("skipping", yolo.layers[i].name)
'''
optimizer = tf.keras.optimizers.Adam()



	 load weights!!!

	 loading complete


In [3]:
from supportFunctions import draw_bbox, postprocess_boxes

In [8]:
def validate_step(counter,image_data, target,orgPath,orgImgName):

    
    image2=tf.squeeze(image_data)

    cv2.imwrite(f'./validation/input.png', image2.numpy()*255)
    
    #input("check111")
    pred_result = yolo(image_data, training=False)


    # optimizing process
    grid = 3 if not TRAIN_YOLO_TINY else 2
    for i in range(grid):
        
        if i>0:
            continue

        '''
            these r 3 feature maps of size 52*52,26*26,13*13,
                                
        '''
        conv, pred = pred_result[i*2], pred_result[i*2+1]

    
#         print("\n\t\t conv pred:",conv.shape)
#         print("\n\t\t pred pred=",pred.shape)
    

        ''' 
            (xc,yc,w,h,obj,pro) is last dimension of size 6

            pred = (1, 13, 13, 3, 6) is reshaped to 
            pred_bbox = (507, 6)   169 is len of single anchor tot 3 vectors             
             #2.pred_result: (1, 52, 52, 2325)
            conv: (1, 26, 26, 18)
            pred = (1, 26, 26, 3, 6)
            pred_bbox = (2028, 6)  676 is lenght of single anchor tot 3 vectors

            image_data= (1, 416, 416, 3)
            pred = (1, 52, 52, 3, 6)
            pred_bbox = (8112, 6)  2704 is len of single anchor tot 3 vectors

        '''

    
        
        pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred]
        #print("\n\t 1.pred_bbox pred=",len(pred_bbox))

        pred_bbox = tf.concat(pred_bbox, axis=0)

        #print("\n\t 2.pred_bbox pred=",pred_bbox.shape)

        ''' 
        this removes invalid boxes like boxes having low confidence or 
        which has size greater than image

        '''
        
        label=target[i][0]
        print("\n\t label shape:",label.shape)
        label=label.reshape(-1,775)
        print("\n\t 2.pred_bbox pred=",pred_bbox.shape,"\t label.shape:",label.shape)
        
        if 0:
            accuracyCalc(pred_bbox,label)
        
        
        if 1:
            
            
            orgImage=cv2.imread(orgPath)
            orgImage2=cv2.imread(orgPath)

            
            #print("\n\t pred_bbox=",len(pred_bbox))
            bboxes = postprocess_boxes(pred_bbox, image2.numpy(),orgImage, YOLO_INPUT_SIZE,0.25)
            #print("\n\t befor nms bboxes =",bboxes.shape)

            if 0:#i==0:
                label=target[i]
                print("\n\t label:",len(label))
                print("\n\t label0:",label[0].shape)
                print("\n\t label0:",label[1].shape)
                #print("\n\t\t\ label",label[])
                abboxes = postprocess_boxes_Anchors(label[0],pred_bbox, image2.numpy(),416,0.25)


            bboxes = nms(bboxes, 0.1, method='nms')

            #print("\n\t after nms 1.bboxes =",len(bboxes))
            #print("\n\t after nms 2.bboxes =",bboxes)
            #draw_bbox() missing 1 required positional argument: 'bboxes

            
            #draw_bbox(nm,counter,level, image, bboxes, CLASSES=YOLO_COCO_CLASSES, show_label=True, show_confidence=True,Text_colors=(255, 255, 0), rectangle_colors='', tracking=False)
            
            #print("\n\t bboxes len:",len(bboxes))
            
            
            resultImage = draw_bboxNew(nm,counter,i,255*image2.numpy(),orgImage,orgImage2, bboxes,orgImgName, CLASSES=YOLO_COCO_CLASSES, rectangle_colors="")
            #resultImage = draw_bbox(nm,counter,i,orgImage, bboxes, CLASSES=YOLO_COCO_CLASSES, rectangle_colors="")

            #cv2.imwrite(f'./validation/{orgImgName}.png',resultImage)
            #cv2.imwrite(f'./validation/{counter}.png',orgImage)

            #input("check")



In [9]:
def draw_bboxNew(nm,counter,level, image,orgImage, bboxes,orgImgName,orgImage2, CLASSES=YOLO_COCO_CLASSES, show_label=True, show_confidence=True,
              Text_colors=(255, 255, 0), rectangle_colors='', tracking=False):
    NUM_CLASS = read_class_names(CLASSES)
    num_classes = len(NUM_CLASS)
    print("\n\t imag shape---:",image.shape)
    
    oh,ow,oc=orgImage.shape
    
    scaleH,scaleW=oh/416,ow/416
    
    print("original image shape:",oh,ow,oc)
    
    try:
        batch,image_h, image_w, _ = image.shape
    except Exception as e:
        batch,image_h, image_w= image.shape


    thickness1 = 3

    for i, bbox in enumerate(bboxes):
        #print("\n\t i=",i)
        coor = np.array(bbox[:4], dtype=np.int32)
        score = bbox[4]
        class_ind = 0#int(bbox[5]) # phosc change
        #print("\n\t score:",score,"\t class_ind:",class_ind)
        print("cor:",coor)
        
        (x1, y1), (x2, y2) = (coor[0], coor[1]), (coor[2], coor[3])
        #(x11, y11), (x22, y22) =(int(x1*scaleW), int(y1*scaleH)), (int(x2*scaleW), int(y2*scaleH))

        x11 = int(x1/(416/ow))
        y11 = int(y1/(416/oh))

        x22 = int(x2/(416/ow))
        y22 = int(y2/(416/oh))

        """
        print("\n\t x1:",x1,"\t y1:",y1,"\t x2:",x2,"\t y2:",y2)
        print("\n\t x11:",x11,"\t y11:",y11,"\t x22:",x22,"\t y22:",y22)
        print(" width:",(ow/416)," height::",(oh/416))
        """
        # print("\n\t image shape:",image.shape)
        # put object rectangle
        #image=cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,0),5)

        image = cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 1)
        #orgImage = cv2.rectangle(orgImage, (x11, y11), (x22, y22), (255,0,0), 3)
        
        #orgImage2=cvTest(x1,y1,x2,y2,orgImage2)
        
        """
        
        y_ = 416
        x_ = 416


        targetSize1 = orgImgName.shape[0]
        targetSize2 = orgImgName.shape[1]

        x_scale = targetSize2 / x_
        y_scale = targetSize1 / y_
        print(x_scale, y_scale)
        img = imageToPredict#cv2.resize(imageToPredict, (targetSize1, targetSize2));
        print("img size:",img.shape)
        img = np.array(img);

        # original frame as named values
        (origLeft, origTop, origRight, origBottom) = (x1,y1,x2,y2)#(160, 35, 555, 470)

        x = int(np.round(origLeft * x_scale))
        y = int(np.round(origTop * y_scale))
        xmax = int(np.round(origRight * x_scale))
        ymax = int(np.round(origBottom * y_scale))
        # Box.drawBox([[1, 0, x, y, xmax, ymax]], img)
        #orgImgName2=drawBox([[1, 0, x, y, xmax, ymax]],orgImgName2)
        
        
        orgImgName2 = cv2.rectangle(orgImgName2, (x, y), (xmax, ymax), (255,0,255), 3)
        
    
        """

        cv2.imwrite(f'./validation/'+orgImgName+'1.png',orgImgName2)

        cv2.imwrite(f'./validation/'+orgImgName+'.png', image)
        cv2.imwrite(f'./validation/'+orgImgName+'_.png', orgImage)

        #input("check!!!")
        
    # Line thickness of 9 px
    # image = cv2.line(image,(0,0),(10,10), color1,5)
    # image = cv2.line(image,(x1,y1),(x2,y2), color1,1)

    if len(bboxes)>10 and counter<10:
        cv2.imwrite(f'./validation/'+orgImgName+'.png', image)
        cv2.imwrite(f'./validation/'+orgImgName+'_.png', orgImage)

    return image



In [10]:
# for counter,(image_data, target,allImagesName) in enumerate(trainset):
from supportFunctions import draw_bbox, postprocess_boxes
imgPaths="/home/aniketag/Documents/phd/yolov5/data//datasets//forms//"
for counter,(image_data, target,allImagesName) in enumerate(testset):
    nm=str(np.random.randint(10000))+".png"
    nm=str(counter)+".png"
    print("\n\t image_data=",image_data.shape)    

    '''
    print("\n\t image_data=",image_data.shape)    
    print("\n\t target:",target[0][0].shape)
    print("\n\t target:",target[1][0].shape)
    print("\n\t target:",target[2][0].shape)
    '''
    print("\n\t allImagesName:",allImagesName)
    
    orgImgName=allImagesName[0].split("forms/")[1]
    orgPath=imgPaths+orgImgName
    validate_step(counter,image_data, target,orgPath,orgImgName)
    #input("check!!!")
    
    if counter==3:
        break


	 image_data= (1, 416, 416, 3)

	 allImagesName: ['/home/k/phd/yolov5/data/datasets/forms/k07-059a.png']

	 label shape: (1, 52, 52, 3, 775)

	 2.pred_bbox pred= (8112, 171) 	 label.shape: (8112, 775)
 inside post process!!!

	 imag shape---: (416, 416, 3)
original image shape: 3542 2479 3
cor: [[204 204 204]
 [204 204 204]
 [208 208 208]
 [208 208 208]]


TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:
import cv2
import numpy as np


def drawBox(boxes, image):
    for i in range(0, len(boxes)):
        # changed color and width to make it visible
        cv2.rectangle(image, (boxes[i][2], boxes[i][3]), (boxes[i][4], boxes[i][5]), (255, 0, 0), 3)
    #cv2.imshow("img", image)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #cv2.imwrite(f'./validation/'+'temp.png', image)
    return image


def cvTest(x1,y1,x2,y2,imageToPredict):
    # imageToPredict = cv2.imread("img.jpg", 3)
    #imageToPredict = cv2.imread("49466033\\img.png ", 3)
    #print(imageToPredict.shape)

    # Note: flipped comparing to your original code!
    # x_ = imageToPredict.shape[0]
    # y_ = imageToPredict.shape[1]
    
    """
    y_ = imageToPredict.shape[0]
    x_ = imageToPredict.shape[1]
    """
    y_ = 416
    x_ = 416
    
    
    targetSize1 = imageToPredict.shape[0]
    targetSize2 = imageToPredict.shape[1]
    
    x_scale = targetSize2 / x_
    y_scale = targetSize1 / y_
    print(x_scale, y_scale)
    img = imageToPredict#cv2.resize(imageToPredict, (targetSize1, targetSize2));
    print("img size:",img.shape)
    img = np.array(img);

    # original frame as named values
    (origLeft, origTop, origRight, origBottom) = (x1,y1,x2,y2)#(160, 35, 555, 470)

    x = int(np.round(origLeft * x_scale))
    y = int(np.round(origTop * y_scale))
    xmax = int(np.round(origRight * x_scale))
    ymax = int(np.round(origBottom * y_scale))
    # Box.drawBox([[1, 0, x, y, xmax, ymax]], img)
    img=drawBox([[1, 0, x, y, xmax, ymax]], img)
    return img

#cvTest()